## Notebook

Add julia functions by calling

In [1]:
include("Julia_function/func_1.jl")

SystemError: SystemError: opening file "c:\\Users\\oyvin\\OneDrive - NTNU\\Documents\\NTNU\\NTNU 8. Semester\\EiT\\optimalisering\\Julia_function\\func_1.jl": No such file or directory

In a julia code env

In [5]:
using Gridap

# Define computational domain
model = CartesianDiscreteModel((0,1,0,1), (4,4))
println("hhh")

hhh
